# Ocean Color

## Import packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

## Define Parameters

In [ ]:
# Time period
start = "1998-01"
stop = "1998-12"

## Define request

In [ ]:
collection_id = "satellite-ocean-colour"

request = {
    "variable": "mass_concentration_of_chlorophyll_a",
    "projection": "regular_latitude_longitude_grid",
    "version": "6_0",
    "format": "zip",
}


requests = download.update_request_date(
    request, start=start, stop=stop, stringify_dates=True
)

## Functions to cache

In [ ]:
def masked_global_mean(ds):
    da = ds["chlor_a"]
    da = da.where((da > 1.0e-3) & (da < 1.0e2))
    with xr.set_options(keep_attrs=True):
        da = 10 ** diagnostics.spatial_weighted_mean(np.log10(da), weights=False)
    da.attrs["long_name"] = da.attrs["long_name"].replace(" (not log-transformed)", "")
    return da.to_dataset()

## Download and transform data

In [ ]:
ds = download.download_and_transform(
    collection_id,
    requests,
    transform_func=masked_global_mean,
    chunks={"year": 1, "month": 1},
)
da_daily = ds["chlor_a"]
da_monthly = da_daily.resample(time="M").mean()
da_seasonality = da_daily.groupby("time.month").mean()

## Daily timeseries

In [ ]:
window = 60
da_daily.plot(label="daily")
da_daily.rolling(time=window, min_periods=1).mean().plot(label=f"{window}-day running")
plt.grid()
_ = plt.legend()

## Monthly timeseries

In [ ]:
window = 6
da_monthly.plot(label="monthly")
da_monthly.rolling(time=window, min_periods=1).mean().plot(
    label=f"{window}-month running"
)
plt.grid()
_ = plt.legend()

## Seasonality

In [ ]:
window = 6
da_seasonality.plot(label="seasonality")
da_seasonality.rolling(month=window, min_periods=1).mean().plot(
    label=f"{window}-month running"
)
plt.title(f"From {start} to {stop}")
plt.grid()
_ = plt.legend()